In [ ]:
# Copyright 2024 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 在 Vertex AI 中使用 Gemeni 開始進行接地

<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/doggy8088/generative-ai/blob/main/gemini/grounding/intro-grounding-gemini.zh.ipynb">
      <img width="32px" src="https://www.gstatic.com/pantheon/images/bigquery/welcome_page/colab-logo.svg" alt="Google Colaboratory 標誌"><br> 在 Colab 中執行
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/colab/import/https:%2F%2Fraw.githubusercontent.com%2Fdoggy8088%2Fgenerative-ai%2Fmain%2Fgemini%2Fgrounding%2Fintro-grounding-gemini.zh.ipynb">
      <img width="32px" src="https://lh3.googleusercontent.com/JmcxdQi-qOpctIvWKgPtrzZdJJK-J3sWE1RsfjZNwshCFgE_9fULcNpuXYTilIR2hjwN" alt="Google Cloud Colab Enterprise 標誌"><br> 在 Colab Enterprise 中執行
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/doggy8088/generative-ai/blob/main/gemini/grounding/intro-grounding.ipynb">
      <img width="32px" src="https://upload.wikimedia.org/wikipedia/commons/9/91/Octicons-mark-github.svg" alt="GitHub 標誌"><br> 在 GitHub 上檢視
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/doggy8088/generative-ai/blob/main/gemini/grounding/intro-grounding-gemini.zh.ipynb">
      <img src="https://www.gstatic.com/images/branding/gcpiconscolors/vertexai/v1/32px.svg" alt="Vertex AI 標誌"><br> 在 Vertex AI Workbench 中開啟
    </a>
  </td>
</table>


| | |
|-|-|
|作者 | [Holt Skinner](https://github.com/holtskinner), [Kristopher Overholt](https://github.com/koverholt) |


**_NOTE_** : 此筆記本已在以下環境中進行了測試：

* Python 版本 = 3.11


## 總覽

利用 [Vertex AI 中的基礎建構](https://cloud.google.com/vertex-ai/docs/generative-ai/grounding/ground-language-models)，你可以利用生成式文字模型，根據自己的文件和資料生成內容基礎。此功能讓模型在執行期間能夠存取的資訊，不侷限於它的訓練資料。透過將模型回應基礎建構在 [Vertex AI Search](https://cloud.google.com/generative-ai-app-builder/docs/enterprise-search-introduction) 中的 Google 搜尋結果或資料儲存，基於資料的 LLM 可以產生更準確、最新的相關回應。

基礎建構提供下列好處：

- 減少模型幻覺 (模型產生不符合事實內容的案例) 
- 將模型回應固定在特定的資訊、文件和資料來源
- 增強產出內容的信賴度、準確度和適用性

在 Vertex AI 的基礎建構情境中，你可以設定兩個不同的基礎來源：

1. 公開取得且已編入索引的資料的 Google 搜尋結果
2. [Vertex AI Search 中的資料儲存](https://cloud.google.com/generative-ai-app-builder/docs/create-datastore-ingest)，其中可透過網站資料、非結構化資料或結構化資料的形式包含你自己的資料

**注意：** 這個範例筆記本中的一些功能需要透過允許名單提早存取某些功能。[透過 Vertex AI Search 進行基礎建構](https://cloud.google.com/vertex-ai/docs/generative-ai/grounding/ground-language-models) 適用於公開預覽，而透過 Google 網路搜尋結果進行基礎建構則適用於私人預覽。如要要求提早存取私人預覽中的功能，請聯絡你的客戶代表或 [Google Cloud 支援團隊](https://cloud.google.com/contact)。


### 目標

在此教學課程中，你將學習如何執行下列操作：

- 產生 LLM 文字和基於 Google 搜尋結果的對話模型回應
- 比較未串接的 LLM 回應與已串接的 LLM 回應的結果
- 在 Vertex AI Search 中建立和使用資料儲存區，以使回應串接到自訂的文件和資料中
- 產生 LLM 文字和基於 Vertex AI Search 結果的對話模型回應

本教學課程使用下列 Google Cloud AI 服務和資源：

- Vertex AI
- Vertex AI Search and Conversation

執行步驟包括：

- 設定 LLM 和提示，以供各項範例使用
- 將範例提示傳送到 Vertex AI 中的生成文字和對話模型
- 在 Vertex AI Search 中建立資料儲存區並使用你自己的資料
- 傳送具有不同程度串接的範例提示 (無串接、網頁串接、資料儲存區串接）


## 在你開始之前

### 設定你的 Google Cloud 專案

**以下步驟為必需，無關於你的筆記本環境。** 

1. [選擇或建立一個 Google Cloud 專案](https://console.cloud.google.com/cloud-resource-manager)。當你第一次建立一個帳戶時，你將會有 300 美元的免費額度可用於你的運算/儲存成本。
1. [確保已為你的專案啟用帳單](https://cloud.google.com/billing/docs/how-to/modify-project)。
1. 啟用 [Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com) 和 [Vertex AI 搜尋與對話 API](https://console.cloud.google.com/flows/enableapi?apiid=discoveryengine.googleapis.com)。
1. 如果你想將落地與 Google 網路搜尋結果結合使用，則你的專案也必須在此功能處於 Private Preview 階段時加入允許清單。
1. 如果你在本機上執行這個筆記本，則你需要安裝 [Cloud SDK](https://cloud.google.com/sdk)。


### 安裝

安裝執行此筆記本所需的下列套件。


In [ ]:
%pip install --upgrade --user --quiet google-cloud-aiplatform==1.42.1

安裝套件後重新啟動 Kernel：


In [ ]:
import IPython

app = IPython.Application.instance()
app.kernel.do_shutdown(True)

<div class="alert alert-block alert-warning">
<b>⚠️ 這個 kernel 將會重新啟動。請等到重新啟動程序完成後再進行下一步。⚠️</b>
</div>


### 設定專案 ID 和地區

**如果你不知道專案 ID** , 請嘗試下列操作：
* 執行 `gcloud config list`。
* 執行 `gcloud projects list`。
* 查看支援頁面：[找出專案 ID](https://support.google.com/googleapi/answer/7014113)

你也可以變更 Vertex AI 使用的 `REGION` 變數。進一步了解 [Vertex AI 地區](https://cloud.google.com/vertex-ai/docs/general/locations)。


In [15]:
PROJECT_ID = "your-project-id"  # @param {type:"string"}
REGION = "us-central1"  # @param {type: "string"}

# Set the project ID
!gcloud auth application-default set-quota-project {PROJECT_ID}
!gcloud config set project {PROJECT_ID}

### 驗證你的 Google Cloud 帳戶

如果你在 Google Colab 上執行此筆記本，你將需要驗證你的環境。為執行此操作，請執行下列新Cell。如果你使用 Vertex AI Workbench，這個步驟並非必要。


In [ ]:
import sys

if "google.colab" in sys.modules:
    # Authenticate user to Google Cloud
    from google.colab import auth

    auth.authenticate_user()

### 匯入函式庫


In [2]:
import vertexai
from vertexai.preview.generative_models import GenerativeModel, Tool, grounding

from IPython.display import display, Markdown

### 初始化 Python 的 Vertex AI SDK

為你的專案初始化 Python 的 Vertex AI SDK：


In [16]:
vertexai.init(project=PROJECT_ID, location=REGION)

初始化 Vertex AI 中的生成式文字和聊天模型：


In [17]:
model = GenerativeModel("gemini-1.0-pro")

## 例子：使用 Google 搜尋結果作為接地

這個例子中，你會將沒有接地的 LLM 回應與接地 Google 搜尋結果後之回應進行比較。你會針對 Google 商店最近發布的硬體提問。


In [5]:
PROMPT = (
    "What are the price, available colors, and storage size options of a Pixel Tablet?"
)

### 無基礎文字生成

向 LLM 發出沒有基礎的預測要求：


In [6]:
response = model.generate_content(PROMPT)

display(Markdown(response.candidates[0].text))

As of my knowledge cutoff in April 2023, the Pixel Tablet has not yet been officially announced or released. Therefore, details such as pricing, color options, and storage size options are not yet available. 

However, you can check the official Google website or reputable tech news sources for the latest and most up-to-date information on the Pixel Tablet once it becomes available.

### 文本產生，依據 Google 搜尋結果

現在你可以加上關鍵字 <Code>tools</Code>，加上一個根據 <Code>grounding.GoogleSearchRetrieval()</Code> 的基礎工具，指示大型語言模型 (LLM) 先使用提示進行 Google 搜尋，然後根據網路搜尋結果建構一個答案：


In [7]:
tool = Tool.from_google_search_retrieval(grounding.GoogleSearchRetrieval())

response = model.generate_content(PROMPT, tools=[tool])

display(Markdown(response.candidates[0].text))

response

- Price: Starting from USD 499
- Storage size options: 128 GB and 256 GB
- Available colors: Porcelain, Rose, and Hazel

candidates {
  content {
    role: "model"
    parts {
      text: "- Price: Starting from USD 499\n- Storage size options: 128 GB and 256 GB\n- Available colors: Porcelain, Rose, and Hazel"
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
  }
  grounding_metadata {
    web_search_queries: "Pixel Tablet price, colors, and storage size options?"
    grounding_attributions {
      web {
        uri: "https://store.google.com/product/pixel_tablet_specs"
        title: "Pixel Tablet Technical Specs - Google Store"
      }
      segment {
        end_index: 119
      }
      confidence_score: 0.778106511
    }
  }
}
usage_metadata {
  prompt_t

請注意，沒有基礎的回應只有來自 LLM 關於 Pixel 平板電腦的有限資訊。而以網路搜尋結果為基礎的回應包含網路搜尋結果的最新資訊，這些資訊會作為帶有基礎要求的 LLM 一部分被返回。


##範例：以客製化文件和資料為基礎

在這個範例中，你會將大型語言模型不依據任何根據的回應與奠基於在 Vertex AI 搜尋 [資料儲存結果](https://cloud.google.com/generative-ai-app-builder/docs/create-datastore-ingest) 的回應做比較。你會詢問一個關於建立 [BigQuery 中的物件資料表](https://cloud.google.com/bigquery/docs/object-table-introduction) 的 GoogleSQL 查詢問題。


### 在 Vertex AI Search 中建立資料庫

按照 [Vertex AI Search 入門文件](https://cloud.google.com/generative-ai-app-builder/docs/try-enterprise-search#create_a_search_app_for_website_data) 中的步驟，建立包含範例資料的 Vertex AI Search 資料庫。在此範例中，你將使用一個網站資料庫，其中包含 Google Cloud 網站上的內容，包含文件。

注意：資料庫必須與你用於 Gemini 的專案相同。

一旦建立完資料庫後，取得資料庫 ID 並在下方輸入。


In [7]:
DATA_STORE_PROJECT_ID = PROJECT_ID  # @param {type:"string"}
DATA_STORE_REGION = "global"  # @param {type:"string"}
# Replace this with your data store ID from Vertex AI Search
DATA_STORE_ID = "your-data-store-id_1234567890123"  # @param {type:"string"}

現在，可以針對 BigQuery 中的物件表格提出問題以及在什麼時候使用它們：


In [6]:
PROMPT = "When should I use an object table in BigQuery? And how does it store data?"

### 無基礎文字生成

向 LLM 發出沒有基礎的預測要求：


In [8]:
response = model.generate_content(PROMPT)

display(Markdown(response.candidates[0].text))

**When to Use an Object Table:**

Use an object table in BigQuery when you need to store and query structured data in a semi-nested format, particularly when:

* You have data with complex or hierarchical relationships.
* You want to efficiently query data across multiple levels of nesting.
* You need to represent parent-child relationships in your data.
* You want to model data that naturally fits into a hierarchical or tree-like structure.

**How Object Tables Store Data:**

Object tables store data as a collection of objects. Each object can have a unique set of fields and can be nested within other objects. The data structure is represented using a JSON-like format called **Arrow Flight Schema**.

An object table can have the following elements:

* **Root Object:** The top-level object in the table.
* **Nested Objects:** Objects within other objects, representing hierarchical relationships.
* **Primitive Fields:** Fields that store basic data types (e.g., string, number, date).
* **Complex Fields:** Fields that contain nested objects or arrays of objects.
* **Arrays:** Ordered lists of objects or primitive values.

**Example:**

Consider the following data representing employees and their departments:

```json
{
  "employees": [
    {
      "name": "Alice",
      "department": {
        "name": "Engineering",
        "manager": "Bob"
      }
    },
    {
      "name": "Bob",
      "department": {
        "name": "Management",
        "manager": "Carol"
      }
    }
  ]
}
```

This data can be represented in an object table as follows:

* **Root Object:** `employees` (an array of employee objects)
* **Nested Object:** `department` (an object representing the department for each employee)
* **Primitive Fields:** `name` (employee name), `manager` (department manager)
* **Complex Field:** `department` (contains nested objects)
* **Array:** `employees` (a collection of employee objects)

By using an object table, you can easily query data across different levels of nesting, such as finding all employees in the "Engineering" department or the manager of "Bob".

### 文字生成依據於 Vertex AI 搜尋結果

現在我們可以加入 `tools` 關鍵字引數，並使用 `grounding.VertexAISearch()` 作為基礎接地工具，指示 LLM 首先在你的自訂資料儲存區中執行搜尋，然後根據相關文件建構答案：


In [9]:
datastore = f"projects/{DATA_STORE_PROJECT_ID}/locations/{DATA_STORE_REGION}/collections/default_collection/dataStores/{DATA_STORE_ID}"
tool = Tool.from_retrieval(
    grounding.Retrieval(grounding.VertexAISearch(datastore=datastore))
)

response = model.generate_content(PROMPT, tools=[tool])

display(Markdown(response.candidates[0].text))

response

Here are the scenarios when you should use an object table in BigQuery:

* **When you need to analyze unstructured data in Cloud Storage.** 
Object tables let you analyze unstructured data in Cloud Storage, such as images, videos, and audio files. 
* **When you need to perform analysis with remote functions.** 
Object tables let you perform analysis with remote functions, such as image classification and speech recognition. 
* **When you need to perform inference by using BigQuery ML.** 
Object tables let you perform inference by using BigQuery ML, such as predicting the sentiment of text or the genre of music.

BigQuery object tables store data in Cloud Storage. The data is stored in a columnar format, which makes it efficient for querying. Object tables also support partitioning, which can improve query performance.

candidates {
  content {
    role: "model"
    parts {
      text: "Here are the scenarios when you should use an object table in BigQuery:\n\n* **When you need to analyze unstructured data in Cloud Storage.** \nObject tables let you analyze unstructured data in Cloud Storage, such as images, videos, and audio files. \n* **When you need to perform analysis with remote functions.** \nObject tables let you perform analysis with remote functions, such as image classification and speech recognition. \n* **When you need to perform inference by using BigQuery ML.** \nObject tables let you perform inference by using BigQuery ML, such as predicting the sentiment of text or the genre of music.\n\nBigQuery object tables store data in Cloud Storage. The data is stored in a columnar format, which makes it efficient for querying. Object tables also support partitioning, which can improve query performance."
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
 

注意，不具備基礎的回應僅來自於 LLM 關於 BigQuery 物件資料表的有限資訊，這些資訊可能不正確。而植基於 Vertex AI Search 結果的回應，則包含來自 Google Cloud 說明文件關於 BigQuery 的最新資訊，以及一些資訊的引文。


## 範例：實際對話回應

你也可以使用 grounding 機制，在 Vertex AI 中使用對話模型時。在本範例中，你將比較沒有 grounding 機制的 LLM 回應，以及 grounding 到 Google 搜尋結果和 Vertex AI Search 中資料儲存所產生。

你會詢問一個有關 Vertex AI 的問題，和一個後續關於 Vertex AI 中管理資料集的問題：


In [10]:
PROMPT = "What are managed datasets in Vertex AI?"
PROMPT_FOLLOWUP = "What types of data can I use?"

### 無接地的聊天會話

開始聊天會話，並向無接地的 LLM 傳送訊息：


In [12]:
chat = model.start_chat()

response = chat.send_message(PROMPT)
display(Markdown(response.text))

response = chat.send_message(PROMPT_FOLLOWUP)
display(Markdown(response.text))

Managed datasets in Vertex AI are scalable, high-performance datasets that can be easily accessed and managed by AI and machine learning models. These datasets are hosted on Google Cloud Storage and are automatically managed by Vertex AI. 

Managed datasets are designed to simplify the process of working with large and complex datasets. This allows data scientists and machine learning engineers to focus on building and training models, rather than managing the underlying infrastructure.

Managed datasets also provide a secure and scalable way to share datasets with collaborators. With managed datasets, users can grant access to specific users or groups, and can easily revoke access when needed. 

Overall, managed datasets are a valuable tool for data scientists and machine learning engineers who are looking to simplify the process of working with large and complex datasets.

You can use a wide variety of data types with managed datasets in Vertex AI, including:

* **Structured data:** This type of data is organized into rows and columns, and can be easily imported from CSV, JSON, or BigQuery files. Examples of structured data include customer data, sales data, and financial data.
* **Unstructured data:** This type of data is not organized into a specific format, and can include text, images, audio, and video. Examples of unstructured data include social media posts, product reviews, and medical records.
* **Semi-structured data:** This type of data has some structure, but is not as rigidly organized as structured data. Examples of semi-structured data include JSON and XML files.

Managed datasets also support a variety of data formats, including:

* CSV
* JSON
* BigQuery
* Apache Parquet
* Apache Avro

You can also use managed datasets to import data from Google Cloud Storage, BigQuery, and other data sources.

Once you have imported your data into a managed dataset, you can use it to train and evaluate machine learning models. Managed datasets are designed to be scalable and high-performance, so you can use them to train even the most complex models.

Here are some examples of how you can use managed datasets in Vertex AI:

* Train a machine learning model to predict customer churn using structured data from a CRM system.
* Train a machine learning model to classify images of products using unstructured data from a product catalog.
* Train a machine learning model to translate text from one language to another using semi-structured data from a translation corpus.

Managed datasets are a versatile and powerful tool that can be used to train and evaluate machine learning models on a wide variety of data types and formats.

### 對話會話根據 Google 搜尋結果而定

現在你可以加上 `tools` 關鍵字參數，使用 `grounding.GoogleSearchRetrieval()` 定位工具指示聊天機器人模型，首先使用提示進行 Google 搜尋，然後根據網頁搜尋結果建構答案：


In [20]:
chat = model.start_chat()
tool = Tool.from_google_search_retrieval(grounding.GoogleSearchRetrieval())

response = chat.send_message(PROMPT, tools=[tool])
display(Markdown(response.text))
print(response)

response = chat.send_message(PROMPT_FOLLOWUP, tools=[tool])
display(Markdown(response.text))
print(response)

Managed datasets in Vertex AI are a fully managed data service that provides a central repository for your training and prediction data. It offers features like data versioning, schema management, and data quality monitoring. Managed datasets also simplify data sharing and collaboration within your organization.

candidates {
  content {
    role: "model"
    parts {
      text: "Managed datasets in Vertex AI are a fully managed data service that provides a central repository for your training and prediction data. It offers features like data versioning, schema management, and data quality monitoring. Managed datasets also simplify data sharing and collaboration within your organization."
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
  }
  grounding_metadata {
    web_search_queries: "What are managed datasets in Vertex AI?"
  }
}
usage_metadata {
  prompt_token_count: 8
  candidates_token_count: 51
  total_token_count: 59
}



Managed datasets in Vertex AI support the following data types:

* **Tabular data:** Data in a tabular format, such as CSV or BigQuery tables.
* **Image data:** Images in JPEG, PNG, or GIF format.
* **Video data:** Videos in MP4 or MOV format.
* **Text data:** Text files in TXT or JSON format.

candidates {
  content {
    role: "model"
    parts {
      text: "Managed datasets in Vertex AI support the following data types:\n\n* **Tabular data:** Data in a tabular format, such as CSV or BigQuery tables.\n* **Image data:** Images in JPEG, PNG, or GIF format.\n* **Video data:** Videos in MP4 or MOV format.\n* **Text data:** Text files in TXT or JSON format."
    }
  }
  finish_reason: STOP
  safety_ratings {
    category: HARM_CATEGORY_HATE_SPEECH
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_DANGEROUS_CONTENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_HARASSMENT
    probability: NEGLIGIBLE
  }
  safety_ratings {
    category: HARM_CATEGORY_SEXUALLY_EXPLICIT
    probability: NEGLIGIBLE
  }
  grounding_metadata {
    web_search_queries: "Vertex AI managed datasets"
  }
}
usage_metadata {
  prompt_token_count: 67
  candidates_token_count: 76
  total_token_count: 143
}



### 在 Vertex AI 搜尋結果中進行接地的聊天會話

現在，我們可以在 `tools` 關鍵字引數中加入 `grounding.VertexAISearch()` 接地工具，指示聊天模型先在自訂資料儲存區中進行搜尋，然後根據相關文件建構答案：


In [21]:
chat = model.start_chat()
datastore = f"projects/{DATA_STORE_PROJECT_ID}/locations/{DATA_STORE_REGION}/collections/default_collection/dataStores/{DATA_STORE_ID}"
tool = Tool.from_retrieval(
    grounding.Retrieval(grounding.VertexAISearch(datastore=datastore))
)

response = chat.send_message(PROMPT, tools=[tool])
display(Markdown(response.text))
print(response)

response = chat.send_message(PROMPT_FOLLOWUP, tools=[tool])
display(Markdown(response.text))
print(response)

Managed datasets in Vertex AI are datasets that are stored and managed by Google Cloud. This means that you don't have to worry about the underlying infrastructure or the management of the data. You can simply access the data through the Vertex AI API or UI.

Managed datasets are a good option for those who want to focus on building and training models without having to worry about the data management. They are also a good option for those who need to share data with others in a secure and controlled way.

Here are some of the benefits of using managed datasets in Vertex AI:

* **Reduced operational overhead:** You don't have to worry about the underlying infrastructure or the management of the data.
* **Improved security:** Managed datasets are stored in a secure and controlled environment.
* **Increased collaboration:** You can easily share data with others in a secure and controlled way.
* **Simplified data management:** You can easily manage your data through the Vertex AI API or UI.

If you are interested in using managed datasets in Vertex AI, you can learn more here: https://cloud.google.com/vertex-ai/docs/datasets/overview

candidates {
  content {
    role: "model"
    parts {
      text: "Managed datasets in Vertex AI are datasets that are stored and managed by Google Cloud. This means that you don\'t have to worry about the underlying infrastructure or the management of the data. You can simply access the data through the Vertex AI API or UI.\n\nManaged datasets are a good option for those who want to focus on building and training models without having to worry about the data management. They are also a good option for those who need to share data with others in a secure and controlled way.\n\nHere are some of the benefits of using managed datasets in Vertex AI:\n\n* **Reduced operational overhead:** You don\'t have to worry about the underlying infrastructure or the management of the data.\n* **Improved security:** Managed datasets are stored in a secure and controlled environment.\n* **Increased collaboration:** You can easily share data with others in a secure and controlled way.\n* **Simplified da

With managed datasets in Vertex AI, you can use the following types of data:

* Tabular data
* Image data
* Video data
* Text data
* Time series data

You can also use managed datasets to store and manage data that is used for training and deploying models. This data can include:

* Training data
* Test data
* Validation data
* Model artifacts
* Predictions

Managed datasets are a good option for storing and managing data that is used for machine learning because they are:

* **Secure:** Data is stored in a secure and controlled environment.
* **Scalable:** Datasets can be scaled to accommodate large amounts of data.
* **Flexible:** Datasets can be used for a variety of machine learning tasks.

To learn more about managed datasets in Vertex AI, you can visit the following link: https://cloud.google.com/vertex-ai/docs/datasets/overview

candidates {
  content {
    role: "model"
    parts {
      text: "With managed datasets in Vertex AI, you can use the following types of data:\n\n* Tabular data\n* Image data\n* Video data\n* Text data\n* Time series data\n\nYou can also use managed datasets to store and manage data that is used for training and deploying models. This data can include:\n\n* Training data\n* Test data\n* Validation data\n* Model artifacts\n* Predictions\n\nManaged datasets are a good option for storing and managing data that is used for machine learning because they are:\n\n* **Secure:** Data is stored in a secure and controlled environment.\n* **Scalable:** Datasets can be scaled to accommodate large amounts of data.\n* **Flexible:** Datasets can be used for a variety of machine learning tasks.\n\nTo learn more about managed datasets in Vertex AI, you can visit the following link: https://cloud.google.com/vertex-ai/docs/datasets/overview"
    }
  }
  finish_reason: STOP
  safety_ratings {
    categor

## 清除

要避免因這個筆記本使用的資源而向你的 Google Cloud 帳戶收費，請執行以下步驟：

1. 要避免不必要的 Google Cloud 費用，請使用 [Google Cloud 控制台](https://console.cloud.google.com/) 刪除你不需要的專案。如需進一步瞭解，請參閱 Google Cloud 的記錄，了解 [如何管理並刪除專案](https://cloud.google.com/resource-manager/docs/creating-managing-projects)。
1. 如果你使用現有的 Google Cloud 專案，請刪除你建立的資源，以避免向你的帳戶收費。如需進一步瞭解，請參閱 [從 Vertex AI 搜尋資料儲存刪除資料](https://cloud.google.com/generative-ai-app-builder/docs/delete-datastores) 的記錄，然後刪除你的資料儲存。
1. 在 Google Cloud 控制台中停用 [Vertex AI 搜尋和對話 API](https://pantheon.corp.google.com/apis/api/discoveryengine.googleapis.com) 和 [Vertex AI API](https://pantheon.corp.google.com/apis/api/aiplatform.googleapis.com)。
